# Exploration of {Guidance}:
* Llama-index guidance pydantic program tutorial
* Microsoft guidance tutorials
* POC to extract information from structured text, news article and scientrific article into structured output (json)

---
CONCLUSIONS/COMMENTS:
* Llama-index guidance pydantic program dicarded due to LLM not outputting correct json schema even with mixtral. # -> OutputParserException: Failed to parse pydantic object from guidance program. Probably the LLM failed to produce data with right json schema. Most probably caused by truncation of output. max_len och max_tokens should be set higher, this was however not investigated in the current notebook.
* Extract data to json works well for select from a list of choices and generation funcion with llama-cpp-python==0.2.26 for guidance 0.1.13. However for Guidance 0.1.14 llama-cpp-python 0.2.62 can be used with good results.
* Content of the generated output from longer text or scientific article good quality. Not all instructions in prompt are followed. Output can possibly be improved with prompting: propmpt engineering, chain-of-thought, few-shot examples.
* Fairly good results extracting information from scientific articles and patents. Interestingly order of Field() in pydantic model matters for output. For example summary early might be creating chain-of-thought effect considering that large part of the context window is probably used. 
* For satisfactory results each field has to be described/prompted which is easily done with Pydantic and Guidance. It is also convinient that each Field generated by Guidance can be accesses as a key:value from generated output
* Mistral and Mixtral performed similarly on patents. Using choice prompting or yes/no prompting resulted in approximately same number of mistakes where the best was Mixtral with yes/no prompting with 85% accuracy on choice questions. Good output from both models on free text generation Fields such as 'summary'.
* Mistral ~1 min/patent while Mixtral ~3min/patent.
* Further work should include for example:
    * prompting with choice for LLM to answer it does not have enough information
    * try to only send in summary and claims from patents rather than whole documents to see if performance will improve
---

---
## Llama-index guidance pydantic program
https://docs.llamaindex.ai/en/stable/examples/output_parsing/guidance_pydantic_program/?h=guidance

In [ ]:
from pydantic import BaseModel
from typing import List
from llama_index.program.guidance import GuidancePydanticProgram

In [ ]:
import guidance
mixtral = guidance.models.LlamaCppChat("/home/dorota/models/mixtral-8x7b-instruct-v0.1.Q4_K_M.gguf") # https://guidance.readthedocs.io/en/latest/api.html
# mistral_Q4 = guidance.models.LlamaCppChat("/home/dorota/models/mistral-7b-instruct-v0.2.Q4_K_M.gguf") # https://huggingface.co/TheBloke/Mistral-7B-Instruct-v0.2-GGUF/resolve/main/mistral-7b-instruct-v0.2.Q4_K_M.gguf?download=true
# mistral_Q6 = guidance.models.LlamaCppChat("/home/dorota/models/mistral-7b-instruct-v0.2.Q6_K.gguf") # https://huggingface.co/TheBloke/Mistral-7B-Instruct-v0.2-GGUF/resolve/main/mistral-7b-instruct-v0.2.Q6_K.gguf?download=true

In [3]:
class Book(BaseModel):
    autor: str
    title: str
    pub_year: int


class BookShelf(BaseModel):
    genre: str
    books: List[Book]

In [4]:
program = GuidancePydanticProgram(
    output_cls=BookShelf,
    prompt_template_str=(
        "Generate a bookshelf with 3 books within a genre with an author, a title and a publication year. Using"
        " genre '{{query_str}}' as inspiration. Parse output as json"
    ),
    guidance_llm=lm,
    verbose=True,
)

In [ ]:
output = program(query_str="Science Fiction", tools_str='')

# -> OutputParserException: Failed to parse pydantic object from guidance program. Probably the LLM failed to produce data with right json schema

In [ ]:
output

---
## {Guidance}

https://github.com/guidance-ai/guidance/blob/main/notebooks/tutorials/intro_to_guidance.ipynb

In [ ]:
import guidance
from guidance import models, gen

# mistral = guidance.models.LlamaCpp("/home/dorota/models/mistral-7b-instruct-v0.2.Q4_K_M.gguf", n_gpu_layers=-1, n_ctx=4096)
mistral = guidance.models.LlamaCpp("/home/dorota/models/mistral-7b-instruct-v0.2.Q6_K.gguf", n_gpu_layers=-1)

In [ ]:
lm = mistral + "Who won the last Kentucky derby and by how much?"
lm + gen(max_tokens=20)

In [ ]:
mistral + '''\
Q: Who won the last Kentucky derby and by how much?
A:''' + gen(stop="Q:")

# -> expected answer

In [ ]:
# using f strings and naming elements of the output

query = "Who won the last Kentucky derby and by how much?"
lm = mistral + f'''\
Q: {query}
A: {gen(name='answer', stop="Q:")}'''

# -> expected answer

lm['answer']

In [ ]:
# function encapsulation

@guidance
def qa_bot(lm, query):
    lm = lm + f'''\
    Q: {query}
    A: {gen(name="answer", stop="Q:")}'''
    return lm

query = "Who won the last Kentucky derby and by how much?"
mistral + qa_bot(query) # note we don't pass the `lm` arg here (that will get passed during execution when it gets added to the model)

In [ ]:
# selecting among alternatives

from guidance import select

query = "Who won the last Kentucky derby and by how much?"

mistral + f'''\
Q: {query}
Now I will choose to either SEARCH the web or RESPOND.
Choice: {select(["SEARCH", "RESPOND"], name="choice")}''' 

In [ ]:
@guidance
def qa_bot(lm, query):
    lm += f'''\
    Q: {query}
    Now I will choose to either SEARCH the web or RESPOND.
    Choice: {select(["SEARCH", "RESPOND"], name="choice")}
    '''
    if lm["choice"] == "SEARCH":
        lm += "A: I don't know, Google it!"
    else:
        lm += f'A: {gen(stop="Q:", name="answer")}'
    return lm

query = "Who won the last Kentucky derby and by how much?"
mistral + qa_bot(query)

In [ ]:
# generating lists

query = "Who won the last Kentucky derby and by how much?"

lm = mistral + f'''\
Q: {query}
Now I will choose to either SEARCH the web or RESPOND.
Choice: {select(["SEARCH", "RESPOND"], name="choice")}
'''
if lm["choice"] == "SEARCH":
    lm += "Here are 3 search queries:\n"
    for i in range(3):
        lm += f'''{i+1}. "{gen(stop='"', name="queries", temperature=1.0, list_append=True)}"\n'''

# Native guidance with mistral_Q6 issues with lists probably due to quantatization of model.

---
## Guidance -> json
https://github.com/guidance-ai/guidance/blob/main/notebooks/guaranteeing_valid_syntax.ipynb \
https://github.com/guidance-ai/guidance?tab=readme-ov-file#guidance-acceleration

In [ ]:
import guidance
from guidance import models, gen, select

lm = guidance.models.LlamaCpp("/home/dorota/models/mistral-7b-instruct-v0.2.Q6_K.gguf", n_gpu_layers=-1)

In [ ]:
# we can pre-define valid option sets
sample_weapons = ["sword", "axe", "mace", "spear", "bow", "crossbow"]
sample_armor = ["leather", "chainmail", "plate"]

# define a re-usable "guidance function" that we can use below
@guidance
def quoted_list(lm, name, n):
    for i in range(n):
        if i > 0:
            lm += ", "
        lm += '"' + gen(name, list_append=True, stop='"') + '"'
    return lm

@guidance
def generate_character(
    lm,
    character_one_liner,
    weapons: list[str] = sample_weapons,
    armour: list[str] = sample_armor,
    n_items: int = 3
):
    lm += f'''\
    {{
        "description" : "{character_one_liner}",
        "name" : "{gen("character_name", stop='"')}",
        "age" : {gen("age", regex="[0-9]+")},
        "armour" : "{select(armour, name="armor")}",
        "weapon" : "{select(weapons, name="weapon")}",
        "class" : "{gen("character_class", stop='"')}",
        "mantra" : "{gen("mantra", stop='"')}",
        "strength" : {gen("age", regex="[0-9]+")},
        "quest_items" : [{quoted_list("quest_items", n_items)}]
    }}'''
    return lm


generation = lm + generate_character("A quick and nimble fighter")

In [ ]:
import json

gen_json = json.loads(str(generation))
print(f"Loaded json:\n{json.dumps(gen_json, indent=4)}")

In [ ]:
type(generation), type(gen_json)

In [ ]:
generation['weapon']

---
---
## POC extract data -> json

In [ ]:
import guidance
from guidance import models, gen, select
import json

lm = guidance.models.LlamaCpp("/home/dorota/models/mistral-7b-instruct-v0.2.Q6_K.gguf", n_gpu_layers=-1, n_ctx=0, echo=False)

# n_ctx=0 is 16384 (max); default n_ctx=512; echo=False -> query and generation output not displayed; n_gpu_layers=0 CPU only, =-1 GPU only

### Structured input

In [ ]:
recepie = """Ingredients:
• 1 cup all-purpose flozfgvzdfgur
• 3 tablespoons granulated sugar
• 1 teaspoon baking powder
• 1/2 teaspoon baking soda
• 1/2 teaspoon salt
• 1 cup milk
• 1 egg
• 3 tablespoons unsalted butter, melted
• Toppings of your choice (fresh fruit, whipped cream, syrup, etc.)"""

In [ ]:
# intial prompt

@guidance
def extract_alergens(lm,recepie):
    lm += f'''\
    Indicate if the recepie contains alergens. Recepie is deliminated by (start) and (stop). 
    Output 'yes' if alergen can be found or 'no' if algergen can not be found. Generated output should be a string.
    --------------------------------------(start)
    {recepie}
    (stop) -------------------------------------- 
    {{
        "eggs" : "{gen("eggs", stop='"')}",
        "milk" : "{gen("milk", stop='"')}",
        "flour" : "{gen("flour", stop='"')}",
        "soy" : "{gen("soy", stop='"')}",
    }}'''
    return lm


generation = lm + extract_alergens(recepie)

# 1 cup all-purpose flozfgvzdfgur is still interpreted as flour
# 1 cup of cream is still interpreted as milk but 1 cookie is not

In [ ]:
# prompt with few-shot example
@guidance
def extract_alergens(lm,recepie):
    lm += f'''\
    Indicate if the recepie contains ingredients. Recepie is deliminated by (start) and (stop). 
    Output 'yes' if ingredient can be found in recepie or 'no' if ingredient can not be found in recepie.
    This is example output:
     ```json
    {{
        "egg" : "yes",
        "milk" : "yes",
        "sugar" : "yes",
        "sand" : "no"
    }}``
    --------------------------------------(start)
    {recepie}
    (stop) -------------------------------------- 
    ```json
    {{
        "egg" : "{gen("eggs", stop='"')}",
        "milk" : "{gen("milk", stop='"')}",
        "flour" : "{gen("flour", stop='"')}",
        "soy" : "{gen("soy", stop='"')}"
    }}```'''
    return lm


generation = lm + extract_alergens(recepie)

# -> correct output

In [ ]:
@guidance
def extract_alergens(lm, recepie):
    lm += f'''\
    Indicate if the recepie contains alergens. Recepie is deliminated by (start) and (stop).
    --------------------------------------(start)
    {recepie}
    (stop)--------------------------------------
    {{
        "egg" : "{select(options=['yes', 'no'], name='egg')}",
        "milk" : "{select(options=['yes', 'no'], name='milk')}",
        "flour" : "{select(options=['yes', 'no'], name='flour')}",
        "gluten" : "{select(options=['yes', 'no'], name='gluten')}",
        "soy" : "{select(options=['yes', 'no'], name='soy')}",
        "egg_amount" : {gen("egg_amount", regex="[0-9]+")}
        "soy_amount" : {gen("soy_amount", regex="[0-9]+")}
    }}'''
    return lm

generation = lm + extract_alergens(recepie)

In [ ]:
# one shot prompting
@guidance
def extract_alergens(lm, recepie):
    lm += f'''\
    Indicate if the recepie contains alergens. Alergen can for example be egg, milk. Recepie is deliminated by (start) and (stop). Only select 'yes' if the alergen is explicitly mentioned in the recepie.
    --------------------------------------(start)
    {recepie}
    (stop)--------------------------------------
    generated_output:
    {{
        "egg" : "{select(options=['yes', 'no'], name='egg')}",
        "milk" : "{select(options=['yes', 'no'], name='milk')}",
        "flour" : "{select(options=['yes', 'no'], name='flour')}",
        "gluten" : "{select(options=['yes', 'no'], name='gluten')}",
        "soy" : "{select(options=['yes', 'no'], name='soy')}",
        "egg_amount" : {gen("egg_amount", regex="[0-9]+")}
        "soy_amount" : {gen("soy_amount", regex="[0-9]+")}
    }}
    
    Check if generated output is correct. 
    correctd_generated_output:
    {{
        "egg" : "{select(options=['yes', 'no'], name='egg')}",
        "milk" : "{select(options=['yes', 'no'], name='milk')}",
        "flour" : "{select(options=['yes', 'no'], name='flour')}",
        "gluten" : "{select(options=['yes', 'no'], name='gluten')}",
        "soy" : "{select(options=['yes', 'no'], name='soy')}",
        "egg_amount" : {gen("egg_amount", regex="[0-9]+")}
        "soy_amount" : {gen("soy_amount", regex="[0-9]+")}
    }}'''
    return lm

generation = lm + extract_alergens(recepie)

# correct and consistent output

### Text input

In [ ]:
ARTICLE = """ New York (CNN)When Liana Barrientos was 23 years old, she got married in Westchester County, New York.
A year later, she got married again in Westchester County, but to a different man and without divorcing her first husband.
Only 18 days after that marriage, she got hitched yet again. Then, Barrientos declared "I do" five more times, sometimes only within two weeks of each other.
In 2010, she married once more, this time in the Bronx. In an application for a marriage license, she stated it was her "first and only" marriage.
Barrientos, now 39, is facing two criminal counts of "offering a false instrument for filing in the first degree," referring to her false statements on the
2010 marriage license application, according to court documents.
Prosecutors said the marriages were part of an immigration scam.
On Friday, she pleaded not guilty at State Supreme Court in the Bronx, according to her attorney, Christopher Wright, who declined to comment further.
After leaving court, Barrientos was arrested and charged with theft of service and criminal trespass for allegedly sneaking into the New York subway through an emergency exit, said Detective
Annette Markowski, a police spokeswoman. In total, Barrientos has been married 10 times, with nine of her marriages occurring between 1999 and 2002.
All occurred either in Westchester County, Long Island, New Jersey or the Bronx. She is believed to still be married to four men, and at one time, she was married to eight men at once, prosecutors say.
Prosecutors said the immigration scam involved some of her husbands, who filed for permanent residence status shortly after the marriages.
Any divorces happened only after such filings were approved. It was unclear whether any of the men will be prosecuted.
Seven of the men are from so-called "red-flagged" countries, including Egypt, Turkey, Georgia, Pakistan and Mali.
Her eighth husband, Rashid Rajput, was deported in 2006 to his native Pakistan after an investigation by the Joint Terrorism Task Force.
If convicted, Barrientos faces up to four years in prison.  Her next court appearance is scheduled for May 18.
"""

@guidance
def summarize(lm, text):
    lm += f'''\
    Generate a title and summary for the text. Text is deliminated by (start) and (stop). Generated title and summary must be strings.
    ---------------------------------------------------------(start)
    {text}
    (stop)----------------------------------------------------------

    structured_output:
    {{
        "text_title" : "{gen(name="title", stop='"')}",
        "text_summary" : "{gen(name="summary", stop='"')}"
    }}'''
    return lm

generation = lm + summarize(ARTICLE)

print(f'''\
      {{
        "title" : "{generation["title"]}",
        "summary" : "{generation["summary"]}",
      }}
''')

# Output:
      # {
      #   "title" : "Woman Charged with Multiple Marriages in Immigration Scam",
      #   "summary" : "Liana Barrientos, 39, of New York, is facing criminal charges for falsifying marriage license applications during a series of 10 marriages between 1999 and 2010. Prosecutors believe the marriages were part of an immigration scam, with some husbands filing for permanent residence status shortly after the weddings. Barrientos is currently charged with two counts of offering a false instrument for filing and faces up to four years in prison if convicted. Seven of her husbands are from 'red-flagged' countries and one was deported in 2006.",
      # }

# -> correct output


In [ ]:
# write structured output part of response to json string and further to dict
import json

gen_json_string = str(generation).split('structured_output:\n')[-1]
gen_dict = json.loads(gen_json_string)
print(json.dumps(gen_dict, indent=4))

---
---

### pdf article as input

In [6]:
from pypdf import PdfReader 
  
reader = PdfReader('/home/dorota/LLM-diploma-project/00_concept_tests/data/12913_2024_Article_10658.pdf') 
num_pages = len(reader.pages)
TEXT = ""
for page_num in range(num_pages): #change to range(num_pages) for whole document
    page = reader.pages[page_num]  
    TEXT += page.extract_text()

In [2]:
import guidance
from guidance import models, gen, select
import json
from llama_cpp import Llama

model = Llama("/home/dorota/models/mistral-7b-instruct-v0.2.Q6_K.gguf", n_gpu_layers=10, n_ctx=0, echo=False, verbose=False)
lm = guidance.models.LlamaCpp(model=model)

ggml_cuda_init: GGML_CUDA_FORCE_MMQ:   no
ggml_cuda_init: CUDA_USE_TENSOR_CORES: yes
ggml_cuda_init: found 1 CUDA devices:
  Device 0: NVIDIA GeForce RTX 4070 Ti, compute capability 8.9, VMM: yes


### Promting tests

In [ ]:
@guidance
def get_metadata(lm, text):
    lm += f'''\
    Article is delimited by (start) and (stop). Extract title, authors and publication year. Generate 5 keywords based on Abstract, do not use keywords in article. Summarize Abstract. Generate research area.

    (start)
    {text}
    (stop)

    JSON ouput:
    {{
        "title" : "{gen(name="title", stop='"')}",
        "authors" : "{gen(name="authors", stop='"')}",
        "publication_year": {gen("publication_year", regex="[0-9]+")},
        "key_words" : "{gen(name="key_words", stop='"')}",
        "summary" : "{gen(name="summary", stop='"')}",
        "research_area" : "{gen(name="research_area", stop='"')}",
        "quality_score" : "{select(options=['GOOD', 'BAD', 'EXCELLENT'], name="quality")}",
        "quality_reason" : "{gen(name="quality_reason", stop='"')}"
    }}'''
    return lm

generation = lm + get_metadata(TEXT)

print(f'''\
      {{
        "title" : "{generation["title"]}",
        "authors" : "{generation["authors"]}",
        "publication_year": "{generation["publication_year"]}",
        "key_words" : "{generation["key_words"]}",
        "summary" : "{generation["summary"]}",
        "research_area" : "{generation["research_area"]}",
        "quality_score" : "{generation["quality"]}",
        "quality_reason" : "{generation["quality_reason"]}"
      }}
''')

# Output:
      # {
      #   "title" : "Visualization of breast cancer -related protein synthesis from the perspective of bibliometric analysis",
      #   "authors" : "Jiawei Xu, Chengdong Yu, Xiaoqiang Zeng, Weifeng Tang, Siyi Xu, Lei Tang, Yanxiao Huang, Zhengkui Sun, Tenghua Yu",
      #   "publication_year": "2023",
      #   "key_words" : "Breast cancer, Bibliometric analysis, Protein synthesis",
      #   "summary" : "This article provides insights into the research on breast cancer and protein synthesis by conducting a bibliometric analysis of articles published between 2003 and 2022 in the Web of Science database. The analysis reveals a steady increase in publications, with most articles published in oncology or biology-related journals. Keyword analysis identified 'breast cancer,' 'expression,' 'cancer,' 'protein,' and 'translation' as the most commonly researched topics. The research focuses on the relationship between protein expression in breast cancer and tumor development and treatment.",
      #   "research_area" : "Breast cancer research, Protein synthesis, Bibliometric analysis",
      #   "quality_score" : "BAD",
      #   "quality_reason" : "The article is a review and does not contain original research data or findings."
      # }

# -> all outputs ok except for key_words which should be 5 and generated from abstract. intead the model picks the 3 listed key_words with slight but logical modification.

In [ ]:
@guidance
def get_metadata(lm, text):
    lm += f'''\
    Article is delimited by (start) and (stop):
    (start)
    {text}
    (stop)

    JSON ouput:
    {{
        "extract title from article" : "{gen(name="title", stop='"')}",
        "extract authors from article" : "{gen(name="authors", stop='"')}",
        "publication_year": {gen("publication_year", regex="[0-9]+")},
        "generate 5 new key words based on content in Abstract" : "{gen(name="key_words", stop='"')}",
        "generate summary in 3 sentences" : "{gen(name="summary", stop='"')}",
        "generate 1 main research area" : "{gen(name="research_area", stop='"')}",
        "quality_score" : "{select(options=['GOOD', 'BAD', 'EXCELLENT', 'CAN NOT SET SCORE'], name="quality")}",
        "describe reason for chosen quality_score in 1 sentece" : "{gen(name="quality_reason", stop='"')}"
    }}'''
    return lm

generation = lm + get_metadata(TEXT)

print(f'''\
      {{
        "title" : "{generation["title"]}",
        "authors" : "{generation["authors"]}",
        "publication_year": "{generation["publication_year"]}",
        "key_words" : "{generation["key_words"]}",
        "summary" : "{generation["summary"]}",
        "research_area" : "{generation["research_area"]}",
        "quality_score" : "{generation["quality"]}",
        "quality_reason" : "{generation["quality_reason"]}"
      }}
''')


# Output:
        # {
        # "title" : "Visualization of breast cancer -related protein synthesis from the perspective of bibliometric analysis",
        # "authors" : "Jiawei Xu, Chengdong Yu, Xiaoqiang Zeng, Weifeng Tang, Siyi Xu, Lei Tang, Yanxiao Huang, Zhengkui Sun, Tenghua Yu",
        # "publication_year": "2023",
        # "key_words" : "Breast cancer research, Protein synthesis analysis, Bibliometric analysis, Cancer diagnosis, Therapy development",
        # "summary" : "This article presents a bibliometric analysis of research on breast cancer and protein synthesis, revealing a growing interest in the relationship between protein expression and breast cancer development and treatment. The research has been instrumental in breast cancer diagnosis and therapy development. Continued research in this area will provide valuable insights into breast cancer biology and the creation of innovative therapies.",
        # "research_area" : "Breast cancer research and protein synthesis analysis",
        # "quality_score" : "BAD",
        # "quality_reason" : "The article lacks novelty and does not provide significant new insights or findings in the field of breast cancer research and protein synthesis analysis."
        # }

# -> perfect output according to instructions when instructions are given as keys for each field to genereate for the model


### Prompting from Pydantic

In [4]:
from pydantic import BaseModel, Field
from typing import List

class ArticleMetadata(BaseModel):
    title: str = Field(..., description="extract title from article")
    authors: str = Field(..., description="extract authors from article")
    pub_year: int = Field(..., description="extract publication year")
    key_words: str = Field(..., description="generate 5 new key words based on content in Abstract")
    summary: str = Field(..., description="generate summary in 3 sentences")
    research_area: str = Field(..., description="generate 1 main research area described in article")
    quality: str = Field(..., description="select one value from provided examples to define quality of article", examples=['GOOD', 'BAD', 'EXCELLENT', 'CAN NOT SET QUALITY'])
    quality_reason: str = Field(..., description="describe reason for chosen quality_score in 1 sentece")

In [ ]:
ArticleMetadata.model_fields

In [ ]:
@guidance
def get_metadata(lm, text, pydantic_class):
    lm += f'''\
    Article is delimited by (start) and (stop):
    (start)
    {text}
    (stop)

    JSON ouput: {{'''

    for key, value in pydantic_class.model_fields.items():
        if value.examples:
            lm += '"' + value.description + " " + ', '.join(value.examples) + '": "' + select(options=value.examples, name=key) + '",'
        elif value.annotation == str:
            lm += '"' + value.description + '": "' + gen(name=key, stop='"') + '",'
        elif value.annotation == int:
            lm += '"' + value.description + '": "' + gen(name=key, regex="[0-9]+") + '",'
    lm += '}'
    return lm

generation = lm + get_metadata(TEXT, ArticleMetadata)

print(f'''\
      {{
        "title" : "{generation["title"]}",
        "authors" : "{generation["authors"]}",
        "publication_year": "{generation["pub_year"]}",
        "key_words" : "{generation["key_words"]}",
        "summary" : "{generation["summary"]}",
        "research_area" : "{generation["research_area"]}",
        "quality_score" : "{generation["quality"]}",
        "quality_reason" : "{generation["quality_reason"]}"
      }}
''')

# Output as json:
# {
#         "title" : "Visualization of breast cancer -related protein synthesis from the perspective of bibliometric analysis",
#         "authors" : "Jiawei Xu, Chengdong Yu, Xiaoqiang Zeng, Weifeng Tang, Siyi Xu, Lei Tang, Yanxiao Huang, Zhengkui Sun, Tenghua Yu",
#         "publication_year": "2023",
#         "key_words" : "Breast cancer research, Protein synthesis analysis, Bibliometric analysis, Cancer diagnosis, Therapy development",
#         "summary" : "This article presents a bibliometric analysis of research on breast cancer and protein synthesis, revealing a growing interest in the relationship between protein expression and breast cancer development and treatment. The study found that most publications were in oncology or biology-related journals, with the most articles published in Journal of Biological Chemistry, Cancer Research, Proceedings of the National Academy of Sciences of the United States of America, and Oncogene. Keyword analysis showed that 'breast cancer', 'expression', 'cancer', 'protein', and 'translation' were the most commonly researched topics.",
#         "research_area" : "Breast cancer research, specifically the relationship between protein expression and breast cancer development and treatment, is a growing area of interest in the scientific community.",
#         "quality_score" : "GOOD",
#         "quality_reason" : "The article provides a comprehensive bibliometric analysis of research on breast cancer and protein synthesis, using reliable data sources and rigorous analysis methods."
#       }


In [ ]:
@guidance
def get_metadata(lm, text, pydantic_class):
    lm += f'''\
    Article is delimited by (start) and (stop):
    (start)
    {text}
    (stop)

    JSON ouput: {{'''

    for key, value in pydantic_class.model_fields.items():
        if value.examples:
            lm += '"' + value.description + " " + ', '.join(value.examples) + '": "' + select(options=value.examples, name=key) + '",'
        elif value.annotation == str:
            lm += '"' + value.description + '": "' + gen(name=key, stop='"') + '",'
        elif value.annotation == int:
            lm += '"' + value.description + '": "' + gen(name=key, regex="[0-9]+") + '",'
    lm += '}'
    return lm

def output_to_pydantic(pydantic_class, output):
    '''Transorms output from lm/guidance -> dict -> original pydantic model'''
    metadata_dict = {}
    for key in pydantic_class.model_fields.keys():
        metadata_dict[key] = output[key]

    output_pydantic_model = pydantic_class(**metadata_dict)
    return output_pydantic_model


model_output = lm + get_metadata(TEXT, ArticleMetadata)
output_to_pydantic(ArticleMetadata, model_output).model_dump() # model_dump() for visualization

# Output in pydantic in accordance to the query pydantic model (TEXT is front page only or the full article 40001_2023_Article_1364.pdf): 
# {'title': 'Visualization of breast cancer -related protein synthesis from the perspective of bibliometric analysis',
#  'authors': 'Jiawei Xu, Chengdong Yu, Xiaoqiang Zeng, Weifeng Tang, Siyi Xu, Lei Tang, Yanxiao Huang, Zhengkui Sun, Tenghua Yu',
#  'pub_year': 2023,
#  'key_words': 'Breast cancer research, Protein synthesis analysis, Bibliometric analysis, Cancer diagnosis, Therapy development',
#  'summary': "This article presents a bibliometric analysis of research on breast cancer and protein synthesis, revealing a growing interest in the relationship between protein expression and breast cancer development and treatment. The study found that most publications were in oncology or biology-related journals, with the most articles published in Journal of Biological Chemistry, Cancer Research, Proceedings of the National Academy of Sciences of the United States of America, and Oncogene. Keyword analysis showed that 'breast cancer', 'expression', 'cancer', 'protein', and 'translation' were the most commonly researched topics.",
#  'research_area': 'Breast cancer research, specifically the relationship between protein expression and breast cancer development and treatment, is a growing area of interest in the scientific community.',
#  'quality': 'GOOD',
#  'quality_reason': 'The article provides a comprehensive bibliometric analysis of research on breast cancer and protein synthesis, using reliable data sources and rigorous analysis methods.'}

# Perfect utput in pydantic in accordance to query in Pydantic model (TEXT is full article 12913_2024_Article_10658.pdf, 60s):
# {'title': 'Anastrozole for the prevention of breast cancer in high-risk postmenopausal women: cost-effectiveness analysis in the UK and the USA',
#  'authors': 'XiaoXia Wei, Jiaqin Cai, Huiting Lin, Wenhua Wu, Jie Zhuang, Hong Sun',
#  'pub_year': 2024,
#  'key_words': 'cost-effectiveness analysis, breast cancer prevention, high-risk postmenopausal women, anastrozole, UK, USA',
#  'summary': 'This study aimed to evaluate the cost-effectiveness of anastrozole for breast cancer prevention in high-risk postmenopausal women from the perspective of third-party payers in the UK and the USA. The results showed that anastrozole had a cost-effective advantage for breast cancer prevention in high-risk women, and the longer the cycle of the model, the higher the acceptability. The findings may provide a reference for clinical medication for clinicians and patients and the scientific decision-making of national government departments for medical and health care.',
#  'research_area': 'Breast cancer prevention and cost-effectiveness analysis',
#  'quality': 'GOOD',
#  'quality_reason': 'The article is well-written, the research is sound, and the findings are relevant and important to the field of breast cancer prevention and cost-effectiveness analysis.'}

### pdf patent as input

In [3]:
from pypdf import PdfReader 
  
reader = PdfReader('/home/dorota/LLM-diploma-project/00_concept_tests/data/patents/EP2671601A1.pdf') 
num_pages = len(reader.pages)
TEXT = ""
for page_num in range(num_pages): #change to range(num_pages) for whole document
    page = reader.pages[page_num]  
    TEXT += page.extract_text()

In [ ]:
import guidance
from guidance import models, gen, select
import json
from llama_cpp import Llama

# model = Llama("/home/dorota/models/mistral-7b-instruct-v0.2.Q6_K.gguf", n_gpu_layers=10, n_ctx=0, echo=False, verbose=False)
model = Llama("/home/dorota/models/mixtral-8x7b-instruct-v0.1.Q4_K_M.gguf", n_gpu_layers=5, n_ctx=0, echo=False, verbose=True)
lm = guidance.models.LlamaCpp(model=model, echo=False, silent=False)

In [2]:
# definitions according to medical device classification guidelines https://health.ec.europa.eu/system/files/2021-10/mdcg_2021-24_en_0.pdf

from pydantic import BaseModel, Field
from typing import List

class PatentMetadata(BaseModel):
    summary: str = Field(..., description="generate summary of the invention in 1 sentence")
    key_technological_field: str = Field(..., description="list 5 key technological concepts in 1-3 words described in patent")
    novelty_level: str = Field(..., description="select one value from provided examples to define level of novelty of invention", examples=['LOW', 'MEDIUM', 'HIGH'])
    novelty_level_reason: str = Field(..., description="describe reason for chosen novelty_level in 1 sentece")
    medical_device_category: str = Field(...,
                                         description="choose device category from provided examples",
                                         examples=['Clinical chemistry and clinical toxicology devices',
                                                   'Hematology and pathology devices',
                                                   'Immunology and microbiology devices',
                                                   'Anesthesiology devices',
                                                   'Cardiovascular devices',
                                                   'Dental devices',
                                                   'Gastroenterology-urology devices',
                                                   'General and plastic surgery devices',
                                                   'General hospital and personal use devices',
                                                   'Neurological devices',
                                                   'Obstetrical and gynecological devices',
                                                   'Ophthalmic devices',
                                                   'Orthopedic devices',
                                                   'Physical medicine devices',
                                                   'Radiology devices'])
    medical_device_type1: str = Field(...,
                                      description="select if device is 'invasive' or 'non-invasive, where 'invasive' means that any part of device penetrates inside the body",
                                      examples=['invasive', 'non-invasive'])
    medical_device_type2: str = Field(...,
                                      description="select if the device is 'active' or 'non-actve', where 'active' is device the operation of which depends on a source of energy other than that generated by the human body for that purpose, or by gravity, and which acts by changing the density of or converting that energy. 'active' are also all devices containing software",
                                      examples=['active', 'non-active'])
    software: str = Field(...,
                          description="select if the device contains software or is connected to device with software or is 'non-software' thus fully manual",
                          examples=['software', 'non-software'])
    measuring_function: str = Field(...,
                                    description="select if the device has a 'measuring function' meaning it is intended to quantify parameters and the result of the measurement in displayed",
                                    examples=['measuring function', 'no measuring function'])
    intended_user: str = Field(...,
                               description="select if the device requires medically trained personal to be used or if it can be used directly by enduser",
                               examples=['medical personal', 'enduser'])
    
    # NOTE Order of fields matters. For example if 'summary' after all fields with 'examples' the summary is just a list of choices repeated and not actual summary of invention.
    # NOTE: generation takes about 1 min per patent for mistral and 3 min for mixtral


In [2]:
# another way of prompting with yes/no

from pydantic import BaseModel, Field
from typing import List

class PatentMetadata(BaseModel):
    summary: str = Field(..., description="generate summary of the invention in 1 sentence")
    key_technological_field: str = Field(..., description="list 5 key technological concepts in 1-3 words described in patent")
    novelty_level: str = Field(..., description="select one value from provided examples to define level of novelty of invention", examples=['LOW', 'MEDIUM', 'HIGH'])
    novelty_level_reason: str = Field(..., description="describe reason for chosen novelty_level in 1 sentece")
    medical_device_category: str = Field(...,
                                         description="choose device category from provided examples",
                                         examples=['Clinical chemistry and clinical toxicology devices',
                                                   'Hematology and pathology devices',
                                                   'Immunology and microbiology devices',
                                                   'Anesthesiology devices',
                                                   'Cardiovascular devices',
                                                   'Dental devices',
                                                   'Gastroenterology-urology devices',
                                                   'General and plastic surgery devices',
                                                   'General hospital and personal use devices',
                                                   'Neurological devices',
                                                   'Obstetrical and gynecological devices',
                                                   'Ophthalmic devices',
                                                   'Orthopedic devices',
                                                   'Physical medicine devices',
                                                   'Radiology devices'])
    invasive: str = Field(...,
                          description="Is the device'invasive', where 'invasive' means that any part of device penetrates inside the body?",
                          examples=['yes', 'no'])
    active: str = Field(...,
                        description="Is the device 'active' meaning device contains software or device operation depends on a source of energy other than that generated by the human body for that purpose, or by gravity, and which acts by changing the density of or converting that energy?",
                        examples=['yes', 'no'])
    software: str = Field(...,
                          description="Does the device contain software or is connected to a device with software?",
                          examples=['yes', 'no'])
    measuring_function: str = Field(...,
                                    description="Does the device have a 'measuring function' meaning it is intended to quantify parameters and the result of the measurement in displayed?",
                                    examples=['yes', 'no'])
    trained_professional_user: str = Field(...,
                               description="Does the device require trained professional medical personel to be used?",
                               examples=['yes', 'no'])

In [3]:
@guidance
def get_metadata(lm, text, pydantic_class):
    lm += f'''\
    Patent is delimited by (start) and (stop):
    (start)
    {text}
    (stop)

    JSON ouput: {{'''

    for key, value in pydantic_class.model_fields.items():
        if value.examples:
            lm += '"' + value.description + " " + ', '.join(value.examples) + '": "' + select(options=value.examples, name=key) + '",'
        elif value.annotation == str:
            lm += '"' + value.description + '": "' + gen(name=key, stop='"') + '",'
        elif value.annotation == int:
            lm += '"' + value.description + '": "' + gen(name=key, regex="[0-9]+") + '",'
    lm += '}'
    return lm

def output_to_pydantic(pydantic_class, output):
    '''Transorms output from lm/guidance -> dict -> original pydantic model'''
    metadata_dict = {}
    for key in pydantic_class.model_fields.keys():
        metadata_dict[key] = output[key]

    output_pydantic_model = pydantic_class(**metadata_dict)
    return output_pydantic_model

# uncomment if want to run this cell for 1 patent analysis:
# model_output = lm + get_metadata(TEXT, PatentMetadata)
# output_to_pydantic(PatentMetadata, model_output).model_dump()


# ---------------------------------------------------------------------------------
# input is full patent WO2014076653A1
# {'title': 'DRILL AND TAP AND METHOD FOR PREOPERATIVE ASSESSMENT OF BONE QUALITY',
#  'inventors': 'THEORIN, Anders; MERSON, Eleanor; LONGDEN, Howard; CARLSSON, Lennart; BJOORN, Dorota',
#  'filling_date': 14112013,
#  'pub_date': 22052014,
#  'area': 'Bone assessment, Dental implantation, Torque monitoring, Twist drill, Tap design',
#  'summary': 'A twist drill and bone tap that monitor torque while drilling or threading to assess jaw bone quality and a method for accessing bone quality prior to or while tapping into the bone during a dental implantation procedure.',
#  'key_technological_concepts': 'Twist drill, Bone tap, Torque monitoring, Dental implantation, Jaw bone quality assessment',
#  'claim_analysis': 'The patent describes a twist drill and bone tap for assessing jaw bone quality during dental implantation procedures. The twist drill includes a shank with a proximal section and a distal section, a mounting portion in the proximal section adapted to connect with a torque monitoring device, and a drill bit connected to the distal section with a cutting portion having at least one helical flute formed thereon. The drill bit has a cutting portion with a helix angle of between about 45 to about 55 degrees, and a drill point located at the end of the cutting portion. The bone tap includes a shank with opposed ends, a mounting portion in one end adapted to connect with a torque monitoring device, and a cutting portion in the other end with at least one helical thread formed thereon. The medial portion between the cutting portion and the mounting portion has a diameter less than the diameter of the cutting portion to minimize friction forces. The method for assessing bone quality involves providing a drill with a shank, a mounting portion, and a drill bit, drilling into the bone, and quantifying the density of the bone based on the measured torque to determine the quality of the bone for implantation. The drill bit has a helix angle of between about 45 to about 55 degrees, and the drill point has a helix angle of about 50 degrees and a point angle of about 130 degrees. The tap has a tapping length of approximately 10 mm and a cutting portion comprised of approximately the first four threads. The method involves inserting a few threads of the cutting portion into a predrilled pilot bore of the bone, incrementally tapping the cutting portion into the hole, and quantifying the density of the bone based on the measured torque to determine the quality of the bone for implantation. The tap has a non-cutting, steering portion located at the end of the shank. The method further involves comparing the measured torque to predetermined torque values of classified bone qualities. The patent also describes a method for assessing bone quality during tapping using a tap with a shank, a mounting portion, a cutting portion, and a medial, non-cutting portion, and a method for assessing bone quality from cutting forces while drilling into bone prior to and during an implantation procedure using a drill with a shank, a mounting portion, and a drill bit.',
#  'novelty': 'The novelty of the patent lies in the use of a twist drill and bone tap that monitor torque while drilling or threading to assess jaw bone quality and a method for accessing bone quality prior to or while tapping into the bone during a dental implantation procedure.',
#  'previous_knowledge': 'The patent does not describe a new material or substance, but rather utilizes existing tools and methods to assess jaw bone quality during dental implantation procedures.',
#  'novelty_level': 'MEDIUM',
#  'novelty_level_reason': 'The invention builds upon existing tools and methods in the field of dental implantation, but offers a novel approach to assessing jaw bone quality by monitoring torque during drilling or threading.'}

#---------------------------------------------------------------------------------------------
# input is full patent EP 2 671 601 A1
# {'number': 'EP 2 671 601 A1',
#  'title': 'An irrigation system with pre-loaded irrigation liquid reservoir',
#  'inventors': '[Andréen, Erik, 417 57 Goteborg (SE); Andersson, Fredrik, 411 25 Goteborg (SE)]',
#  'filling_date': 7062012,
#  'pub_date': 11122013,
#  'area': 'Irrigation system, Pre-loaded reservoir, Rectal irrigation, Self-administration, Control unit',
#  'summary': 'An irrigation system comprising a pre-loaded reservoir for an irrigating liquid, a probe for arrangement in a user, and a control unit for controlling the transfer of said irrigation liquid, wherein the reservoir is arranged to exert a mechanical pressure on the irrigation liquid therein, thereby maintaining the irrigation liquid under pressure, and wherein the control unit comprises a control valve for releasing irrigation liquid from the reservoir for discharge through the probe.',
#  'key_technological_concepts': 'Irrigation system, Pre-loaded reservoir, Rectal irrigation, Self-administration, Control unit, Mechanical pressure, Pressure maintenance, Control valve',
#  'claim_analysis': 'The patent describes an irrigation system comprising a reservoir for an irrigating liquid, a probe for arrangement in a user, and a control unit for controlling the transfer of said irrigation liquid. The reservoir is arranged to exert a mechanical pressure on the irrigation liquid therein, thereby maintaining the irrigation liquid under pressure, and the control unit comprises a control valve for releasing irrigation liquid from the reservoir for discharge through the probe. The reservoir may be made of a rigid or semi-rigid material, and have at least one moveable wall and a retention element to exert a force to retain the moveable wall in a contracted position. Alternatively, the reservoir may comprise an elastically stretchable container, which when elastically stretched exerts a mechanical pressure on the irrigation liquid therein. The probe may comprise an inflatable balloon for fixing the catheter in a body cavity, and the control unit may be arranged to control the fluid for the balloon. The control unit may further comprise a second control valve for releasing irrigation liquid from the reservoir for inflation of the inflatable balloon, and a third control valve for releasing fluid from the balloon when inflated. The irrigation system may further comprise a pump arranged for pumping a fluid to the balloon for inflation, and a volume indicator arranged within the reservoir. The irrigation system is suitable for rectal irrigation and self-administration. The patent also mentions that the reservoir may be loaded by the hydraulic pressure in common water distribution systems, or by mechanical compression means, or by a gas cartridge. The valves and the controls for the valves may be realized in many different ways, and the pumping of the irrigation liquid may be direct or indirect. The mechanical pressurization of the reservoir may be achieved by many different types of springs, elastic means, and the like. The valves may also allow setting of intermediate opening states, in a stepped or continuous fashion. The patent also mentions that the irrigation liquid can be any liquid which is capable of irrigation the body cavity of interest, including water, hypertonic aqueous salt solutions, solutions or suspensions of cathartic agents, and mineral oil. The patent also mentions that the irrigation system can be used for carrying out anal irrigation by filling the reservoir with a liquid, connecting the reservoir and the probe to the control unit, inserting the probe into the rectum, inflating the balloon for fixing the probe in the body, transferring irrigation liquid from the reservoir to the probe for irrigation, and deflating and removing the probe after irrigation is completed.',
#  'novelty': 'An irrigation system with a pre-loaded reservoir for an irrigating liquid, a probe for arrangement in a user, and a control unit for controlling the transfer of said irrigation liquid, wherein the reservoir is arranged to exert a mechanical pressure on the irrigation liquid therein, thereby maintaining the irrigation liquid under pressure, and wherein the control unit comprises a control valve for releasing irrigation liquid from the reservoir for discharge through the probe, is provided.',
#  'previous_knowledge': 'The patent describes an irrigation system for rectal irrigation, but does not provide a novel design or method for the irrigation system itself, only improvements to the control and pressure maintenance aspects of the system.',
#  'novelty_level': 'MEDIUM',
#  'novelty_level_reason': 'The patent describes improvements to the control and pressure maintenance aspects of an existing irrigation system, making it more convenient and efficient for self-administration of rectal irrigation, but does not provide a novel design or method for the irrigation system itself.'}


In [ ]:
# run all patents in specified folder
import os
from pypdf import PdfReader 

directory = '/home/dorota/LLM-diploma-project/00_concept_tests/data/patents'
for filename in os.listdir(directory):
    file = os.path.join(directory, filename)
    print(filename)
    
    reader = PdfReader(file) 
    num_pages = len(reader.pages)
    TEXT = ""
    for page_num in range(num_pages):
        page = reader.pages[page_num]  
        TEXT += page.extract_text()
    
    model_output = lm + get_metadata(TEXT, PatentMetadata)
    pydantic_output = output_to_pydantic(PatentMetadata, model_output)
    for key, value in pydantic_output.model_dump().items():
        print(f"{key}: {value}")
        
    print("\n--------------------------------------------------")


In [ ]:
# # model = mistral 1 min/patent

# EP2679259A1.pdf
# summary: A portable irrigation system comprising a reservoir for an irrigating liquid, a probe for arrangement in a user, an electrical system including an electrical pump for directly or indirectly pumping irrigation liquid from the reservoir to the probe, a battery for driving the electrical pump, at least one control switch for activation of the electrical pump, and wiring connecting the parts, a control unit for controlling the transfer of irrigation liquid, tubing providing fluid communication between the reservoir, control unit and probe, a waterproof housing enclosing the battery, and the electrical system being galvanically isolated from the exterior and chargeable through inductive charging.
# key_technological_field: Portable irrigation system, Galvanic isolation, Inductive charging, Waterproof housing, Electrical pump
# novelty_level: MEDIUM
# novelty_level_reason: The invention provides a portable irrigation system with a waterproof housing, galvanic isolation, and inductive charging, which are known features in the art but are combined in a novel way to improve the system's functionality and user experience.
# medical_device_category: Gastroenterology-urology devices
# medical_device_type1: non-invasive NOTE wrong
# medical_device_type2: active
# software: non-software NOTE wrong
# measuring_function: no measuring function NOTE wrong
# intended_user: enduser

# --------------------------------------------------
# WO2014076653A1.pdf
# summary: A twist drill and bone tap for assessing jaw bone quality during dental implantation procedures, each having a shank with a proximal section and a distal section, a mounting portion formed in the proximal section for connecting with a torque monitoring device, and a cutting portion with at least one helical flute or thread for generating a measurable torque as a function of the bone material being drilled or tapped.
# key_technological_field: Twist drill, Bone tap, Torque monitoring, Helical flute, Measurable torque
# novelty_level: MEDIUM
# novelty_level_reason: The invention involves a twist drill and bone tap for assessing jaw bone quality during dental implantation procedures, which is a common practice in dentistry, but the novelty lies in the use of a shank with a proximal section and a distal section, a mounting portion formed in the proximal section for connecting with a torque monitoring device, and a cutting portion with at least one helical flute or thread for generating a measurable torque as a function of the bone material being drilled or tapped, which allows for objective and quantitative assessment of bone quality prior to or during the implantation procedure.
# medical_device_category: Dental devices
# medical_device_type1: invasive
# medical_device_type2: active
# software: non-software NOTE should be software???
# measuring_function: measuring function
# intended_user: enduser NOTE wrong

# --------------------------------------------------
# EP2671602A1.pdf
# summary: An irrigation system comprising a reservoir for an irrigating liquid, a probe for arrangement in a user, a first pump for directly or indirectly pumping irrigation liquid from the reservoir to the probe, a second pump for pumping a fluid to the balloon for inflation, a control unit for controlling the transfer of the irrigation liquid and the fluid, and tubing providing fluid communication between the reservoir, control unit and probe, wherein the control unit and the first and second pumps are arranged as an integrated unitary component, and the manually operable balloon control element is arranged in the vicinity of the second pump.
# key_technological_field: Irrigation system, Dual pumps, Balloon control, Integrated unitary component, Manually operable
# novelty_level: MEDIUM
# novelty_level_reason: The invention provides an irrigation system with a control unit and dual pumps arranged as an integrated unitary component, and a manually operable balloon control element arranged in the vicinity of the second pump, which is a known concept but the arrangement and integration of the components contribute to improved functionality and ease of use.
# medical_device_category: Gastroenterology-urology devices
# medical_device_type1: non-invasive NOTE wrong
# medical_device_type2: active
# software: non-software
# measuring_function: no measuring function
# intended_user: enduser

# --------------------------------------------------
# EP2783724A1.pdf
# summary: A coupling device for a tubular arrangement for medical use comprises a first connector part and a second connector part, wherein one of the connector parts comprises an engagement/disengagement member and the other of the connector parts comprises a locking element, and the first connector part further comprises a guiding edge protruding from the first connector surface towards the second connector part, and being arranged around the female connecting portions. The second connector part is kept in a position by the guiding edge, and the male connecting portions engage with the female connecting portions when rotating less than one turn of the second connector part. The engagement/disengagement member is rotated to engage the connection parts together.
# key_technological_field: Guiding edge, Engagement/disengagement member, Locking element, Rotatably engage, Medical use
# novelty_level: MEDIUM
# novelty_level_reason: The invention provides a coupling device for medical use with a guiding edge that facilitates engagement and reduces the need for precision, and a locking element that rotatably engages with the engagement/disengagement member to secure the connection. While the concept of a guiding edge and a locking element is not new, the combination and arrangement of these features in a medical coupling device is novel and improves the ease of use and reliability of the device.
# medical_device_category: Gastroenterology-urology devices
# medical_device_type1: non-invasive
# medical_device_type2: non-active
# software: non-software
# measuring_function: no measuring function
# intended_user: enduser

# --------------------------------------------------
# EP3925641A1.pdf
# summary: An irrigation system for rectal irrigation comprises a reusable part and a disposable part, the reusable part comprising a liquid container with a first opening and a first connection interface, and the disposable part comprising a protective cover comprising a tubular part, a one-way valve in the tubular part, and a protective skirt connected to and encircling the tubular part, wherein the tubular part is provided with a second connection interface which is releasably connectable to the first connection interface.
# key_technological_field: Reusable part, Disposable part, Protective cover, One-way valve, Connection interface
# novelty_level: MEDIUM
# novelty_level_reason: The invention provides a new design for an irrigation system with a reusable part and a disposable part, where the disposable part includes a protective cover with a one-way valve and a connection interface that is releasably connectable to the reusable part's connection interface.
# medical_device_category: Gastroenterology-urology devices
# medical_device_type1: non-invasive NOTE wrong
# medical_device_type2: active
# software: non-software
# measuring_function: no measuring function
# intended_user: enduser

# --------------------------------------------------
# EP2671601A1.pdf
# summary: An irrigation system comprising a reservoir for an irrigating liquid, a probe for arrangement in a user, a control unit for controlling the transfer of said irrigation liquid, and tubing providing fluid communication between said reservoir, control unit and probe, wherein said reservoir is arranged to exert a mechanical pressure on the irrigation liquid therein, thereby maintaining the irrigation liquid under pressure, and wherein said control unit comprises a control valve for releasing irrigation liquid from said reservoir for discharge through said probe.
# key_technological_field: Elastic reservoir, Control unit, Probe, Irrigation liquid, Pressure
# novelty_level: MEDIUM
# novelty_level_reason: The invention provides an irrigation system with a pre-loaded irrigation liquid reservoir, which is a common concept, but the system also includes a control unit with a control valve for releasing irrigation liquid, which adds a level of convenience and ease of use for the user.
# medical_device_category: Gastroenterology-urology devices
# medical_device_type1: non-invasive NOTE wrong
# medical_device_type2: active
# software: non-software 
# measuring_function: measuring function
# intended_user: enduser

# --------------------------------------------------
# EP2810669A1.pdf
# summary: A collapsible container for medical use, comprising a flexible side wall, a rigid top portion and a rigid bottom portion, is provided with a flexible tube connected to the bottom portion and the top portion, allowing fluid communication between the interior bottom of the container and a discharge hole of the rigid top portion. The container is reversibly foldable and unfoldable, and can be collapsed by moving the rigid bottom and top portions towards each other and locking them in the compacted state.
# key_technological_field: Collapsible container, Fluid communication, Reversible folding, Medical use, Irrigation system
# novelty_level: MEDIUM
# novelty_level_reason: The invention provides a collapsible container for medical use with a flexible tube for fluid communication, which is reversibly foldable and unfoldable, and can be collapsed by moving the rigid bottom and top portions towards each other and locking them in the compacted state. This concept is known in the art, but the specific arrangement and method of collapsing the container is novel and improves the usability and portability of the container.
# medical_device_category: Gastroenterology-urology devices
# medical_device_type1: non-invasive
# medical_device_type2: active
# software: non-software
# measuring_function: no measuring function
# intended_user: enduser

# --------------------------------------------------
# EP3295977A1.pdf
# summary: An irrigation system comprising a reservoir for an irrigating liquid, a probe for arrangement in a user, tubing providing fluid communication between the reservoir and the probe, an electrical pump for indirectly pumping irrigation liquid from the reservoir to the probe through the tubing, the electrical pump being controllable to assume a plurality of predetermined flow rates, an electrically operable valve operable to continuously control the degree of openness of the tubing between a fully closed state and a fully opened state, a flow sensor to measure an actual flow rate of the irrigation liquid in the probe, and a controller arranged to obtain a desired flow rate from a user interface, and configured to control the flow rate of the electric pump to one of the plurality of flow rates exceeding the desired flow rate, and to continuously regulate the electrically operable valve based on the measured actual flow rate, thereby limiting the flow rate provided by the electric pump to obtain the desired output flow rate.
# key_technological_field: Electrical pump, Flow control, Valve, Sensor, Controller
# novelty_level: MEDIUM
# novelty_level_reason: The invention provides an improved irrigation system with precise flow control and real-time adjustment capabilities, making it more effective and convenient for users.
# medical_device_category: Gastroenterology-urology devices
# medical_device_type1: non-invasive NOTE wrong
# medical_device_type2: active
# software: software
# measuring_function: measuring function
# intended_user: enduser

# --------------------------------------------------



In [ ]:
# # MISTRAL yes/no prompt

# EP2679259A1.pdf
# summary: A portable irrigation system comprising a reservoir for an irrigating liquid, a probe for arrangement in a user, an electrical system including an electrical pump for directly or indirectly pumping irrigation liquid from the reservoir to the probe, a battery for driving the electrical pump, at least one control switch for activation of the electrical pump, and wiring connecting the parts, a control unit for controlling the transfer of irrigation liquid, tubing providing fluid communication between the reservoir, control unit and probe, a waterproof housing enclosing the battery, and the electrical system being galvanically isolated from the exterior and chargeable through inductive charging.
# key_technological_field: Portable irrigation system, Galvanic isolation, Inductive charging, Waterproof housing, Electrical pump
# novelty_level: MEDIUM
# novelty_level_reason: The invention provides a portable irrigation system with a waterproof housing, galvanic isolation, and inductive charging, which are known features in the art but are combined in a novel way to improve the system's functionality and user experience.
# medical_device_category: Gastroenterology-urology devices
# invasive: yes
# active: yes
# software: yes
# measuring_function: yes
# trained_professional_user: no

# --------------------------------------------------
# WO2014076653A1.pdf
# summary: A twist drill and bone tap for assessing jaw bone quality during dental implantation procedures, each having a shank with a proximal section and a distal section, a mounting portion formed in the proximal section for connecting with a torque monitoring device, and a cutting portion with at least one helical flute or thread for generating a measurable torque as a function of the bone material being drilled or tapped.
# key_technological_field: Twist drill, Bone tap, Torque monitoring, Helical flute, Measurable torque
# novelty_level: MEDIUM
# novelty_level_reason: The invention involves a twist drill and bone tap for assessing jaw bone quality during dental implantation procedures, which is a common practice in dentistry, but the novelty lies in the use of a shank with a proximal section and a distal section, a mounting portion formed in the proximal section for connecting with a torque monitoring device, and a cutting portion with at least one helical flute or thread for generating a measurable torque as a function of the bone material being drilled or tapped, which allows for objective and quantitative assessment of bone quality prior to or during the implantation procedure.
# medical_device_category: Dental devices
# invasive: yes
# active: yes
# software: yes
# measuring_function: yes
# trained_professional_user: yes

# --------------------------------------------------
# EP2671602A1.pdf
# summary: An irrigation system comprising a reservoir for an irrigating liquid, a probe for arrangement in a user, a first pump for directly or indirectly pumping irrigation liquid from the reservoir to the probe, a second pump for pumping a fluid to the balloon for inflation, a control unit for controlling the transfer of the irrigation liquid and the fluid, and tubing providing fluid communication between the reservoir, control unit and probe, wherein the control unit and the first and second pumps are arranged as an integrated unitary component, and the manually operable balloon control element is arranged in the vicinity of the second pump.
# key_technological_field: Irrigation system, Dual pumps, Balloon control, Integrated unitary component, Manually operable
# novelty_level: MEDIUM
# novelty_level_reason: The invention provides an irrigation system with a control unit and dual pumps arranged as an integrated unitary component, and a manually operable balloon control element arranged in the vicinity of the second pump, which is a known concept but the arrangement and integration of the components contribute to improved functionality and ease of use.
# medical_device_category: Gastroenterology-urology devices
# invasive: yes
# active: yes
# software: yes NOTE no
# measuring_function: yes NOTE nothing mentioned in patent
# trained_professional_user: no

# --------------------------------------------------
# EP2783724A1.pdf
# summary: A coupling device for a tubular arrangement for medical use comprises a first connector part and a second connector part, wherein one of the connector parts comprises an engagement/disengagement member and the other of the connector parts comprises a locking element, and the first connector part further comprises a guiding edge protruding from the first connector surface towards the second connector part, and being arranged around the female connecting portions. The second connector part is kept in a position by the guiding edge, and the male connecting portions engage with the female connecting portions when rotating less than one turn of the second connector part. The engagement/disengagement member is rotated to engage the connection parts together.
# key_technological_field: Guiding edge, Engagement/disengagement member, Locking element, Rotatably engage, Medical use
# novelty_level: MEDIUM
# novelty_level_reason: The invention provides a coupling device for medical use with a guiding edge that facilitates engagement and reduces the need for precision, and a locking element that rotatably engages with the engagement/disengagement member to secure the connection. While the concept of a guiding edge and a locking element is not new, the combination and arrangement of these features in a medical coupling device is novel and improves the ease of use and reliability of the device.
# medical_device_category: Gastroenterology-urology devices
# invasive: yes NOTE no
# active: yes NOTE no
# software: yes NOTE no
# measuring_function: yes NOTE no
# trained_professional_user: yes NOTE no

# --------------------------------------------------
# EP3925641A1.pdf
# summary: An irrigation system for rectal irrigation comprises a reusable part and a disposable part, the reusable part comprising a liquid container with a first opening and a first connection interface, and the disposable part comprising a protective cover comprising a tubular part, a one-way valve in the tubular part, and a protective skirt connected to and encircling the tubular part, wherein the tubular part is provided with a second connection interface which is releasably connectable to the first connection interface.
# key_technological_field: Reusable part, Disposable part, Protective cover, One-way valve, Connection interface
# novelty_level: MEDIUM
# novelty_level_reason: The invention provides a new design for an irrigation system with a reusable part and a disposable part, where the disposable part includes a protective cover with a one-way valve and a connection interface that is releasably connectable to the reusable part's connection interface.
# medical_device_category: Gastroenterology-urology devices
# invasive: yes
# active: yes
# software: yes NOTE no
# measuring_function: no
# trained_professional_user: no

# --------------------------------------------------
# EP2671601A1.pdf
# summary: An irrigation system comprising a reservoir for an irrigating liquid, a probe for arrangement in a user, a control unit for controlling the transfer of said irrigation liquid, and tubing providing fluid communication between said reservoir, control unit and probe, wherein said reservoir is arranged to exert a mechanical pressure on the irrigation liquid therein, thereby maintaining the irrigation liquid under pressure, and wherein said control unit comprises a control valve for releasing irrigation liquid from said reservoir for discharge through said probe.
# key_technological_field: Elastic reservoir, Control unit, Probe, Irrigation liquid, Pressure
# novelty_level: MEDIUM
# novelty_level_reason: The invention provides an irrigation system with a pre-loaded irrigation liquid reservoir, which is a common concept, but the system also includes a control unit with a control valve for releasing irrigation liquid, which adds a level of convenience and ease of use for the user.
# medical_device_category: Gastroenterology-urology devices
# invasive: yes
# active: yes
# software: yes NOTE no
# measuring_function: yes
# trained_professional_user: no

# --------------------------------------------------
# EP2810669A1.pdf
# summary: A collapsible container for medical use, comprising a flexible side wall, a rigid top portion and a rigid bottom portion, is provided with a flexible tube connected to the bottom portion and the top portion, allowing fluid communication between the interior bottom of the container and a discharge hole of the rigid top portion. The container is reversibly foldable and unfoldable, and can be collapsed by moving the rigid bottom and top portions towards each other and locking them in the compacted state.
# key_technological_field: Collapsible container, Fluid communication, Reversible folding, Medical use, Irrigation system
# novelty_level: MEDIUM
# novelty_level_reason: The invention provides a collapsible container for medical use with a flexible tube for fluid communication, which is reversibly foldable and unfoldable, and can be collapsed by moving the rigid bottom and top portions towards each other and locking them in the compacted state. This concept is known in the art, but the specific arrangement and method of collapsing the container is novel and improves the usability and portability of the container.
# medical_device_category: Gastroenterology-urology devices
# invasive: yes NOTE no
# active: yes
# software: yes NOTE no
# measuring_function: yes NOTE no
# trained_professional_user: yes NOTE no

# --------------------------------------------------
# EP3295977A1.pdf
# summary: An irrigation system comprising a reservoir for an irrigating liquid, a probe for arrangement in a user, tubing providing fluid communication between the reservoir and the probe, an electrical pump for indirectly pumping irrigation liquid from the reservoir to the probe through the tubing, the electrical pump being controllable to assume a plurality of predetermined flow rates, an electrically operable valve operable to continuously control the degree of openness of the tubing between a fully closed state and a fully opened state, a flow sensor to measure an actual flow rate of the irrigation liquid in the probe, and a controller arranged to obtain a desired flow rate from a user interface, and configured to control the flow rate of the electric pump to one of the plurality of flow rates exceeding the desired flow rate, and to continuously regulate the electrically operable valve based on the measured actual flow rate, thereby limiting the flow rate provided by the electric pump to obtain the desired output flow rate.
# key_technological_field: Electrical pump, Flow control, Valve, Sensor, Controller
# novelty_level: MEDIUM
# novelty_level_reason: The invention provides an improved irrigation system with precise flow control and real-time adjustment capabilities, making it more effective and convenient for users.
# medical_device_category: Gastroenterology-urology devices
# invasive: yes
# active: yes
# software: yes
# measuring_function: yes
# trained_professional_user: no

# --------------------------------------------------


In [ ]:
# MIXTRAL 5 min/patent

# EP2679259A1.pdf
# summary: An irrigation system comprising a reservoir for an irrigating liquid, a probe for arrangement in a user, an electrical system, a control unit for controlling the transfer of irrigation liquid, and tubing providing fluid communication between the reservoir, control unit and probe, wherein the control unit comprises a waterproof housing enclosing at least the battery and the electrical system is galvanically isolated from the exterior of the irrigation system, and wherein the battery is chargeable through inductive charging.
# key_technological_field: irrigation system, waterproof housing, galvanic isolation, inductive charging
# novelty_level: MEDIUM
# novelty_level_reason: The irrigation system is not the first of its kind, but it has some unique features such as the waterproof housing and the galvanic isolation of the electrical system, which make it more robust and safe to use. Additionally, the battery is chargeable through inductive charging, which eliminates the need for electrical connections and further increases the safety of the system.
# medical_device_category: Gastroenterology-urology devices
# medical_device_type1: invasive
# medical_device_type2: active
# software: non-software NOTE software
# measuring_function: no measuring function NOTE measuring function
# intended_user: enduser

# --------------------------------------------------
# WO2014076653A1.pdf
# summary: A twist drill and bone tap each monitor torque while drilling or threading to assess jaw bone quality and a method for accessing bone quality prior to or while tapping into the bone during a dental implantation procedure.
# key_technological_field: torque monitoring, dental implantation, bone quality assessment, twist drill, bone tap
# novelty_level: MEDIUM
# novelty_level_reason: The invention is novel in that it provides a twist drill and bone tap that monitor torque while drilling or threading to assess jaw bone quality and a method for accessing bone quality prior to or while tapping into the bone during a dental implantation procedure.
# medical_device_category: Dental devices
# medical_device_type1: invasive
# medical_device_type2: active
# software: non-software NOTE software
# measuring_function: measuring function
# intended_user: medical personal

# --------------------------------------------------
# EP2671602A1.pdf
# summary: An irrigation system comprising a reservoir for an irrigating liquid, a probe for arrangement in a user, a first pump for directly or indirectly pumping irrigation liquid from the reservoir to the probe, a second pump for pumping a fluid to the balloon for inflation, a control unit for controlling the transfer of the irrigation liquid and the fluid, wherein the control unit further comprises a valve for releasing fluid from the balloon for deflation, the valve being provided with a manually operable balloon control element, and tubing providing fluid communication between the reservoir, control unit and probe; wherein the control unit and the first and second pumps are arranged as an integrated unitary component, and the manually operable balloon control element is arranged in the vicinity of the second pump.
# key_technological_field: irrigation system, balloon control, pump, reservoir, tubing
# novelty_level: MEDIUM
# novelty_level_reason: The invention is novel in that it provides an irrigation system with a control unit and pumps arranged as an integrated unitary component, and a manually operable balloon control element arranged in the vicinity of the second pump.
# medical_device_category: Gastroenterology-urology devices
# medical_device_type1: invasive
# medical_device_type2: active
# software: non-software
# measuring_function: no measuring function
# intended_user: enduser

# --------------------------------------------------
# EP2783724A1.pdf
# summary: A coupling device for a tubular arrangement for medical use comprising a first connector part and a second connector part, wherein one of said connector parts comprises an engagement/disengagement member for at least assisting in engaging and disengaging the engagement between said connector parts, and wherein the other of said connector parts comprises a locking element arranged to rotatably engage with the engagement/disengagement member, wherein the first connector part comprises at least two female connecting portions arranged in a first connector surface and a guiding edge protruding from the first connector surface arranged around the at least two female connecting portions, and the second connector part comprises at least two male connecting portions arranged in a second connector surface, and wherein at least one of the first connector part and the second connector part comprises a locking element arranged to rotatably engage with the engagement/disengagement member.
# key_technological_field: coupling device, tubular arrangement, medical use, engagement/disengagement member, locking element
# novelty_level: MEDIUM
# novelty_level_reason: The invention is novel in that it provides a coupling device for a tubular arrangement for medical use with a guiding edge that facilitates engagement of the coupling device, by forming a recess in the connection interface. The guiding edge enables reduction of the need for precision when engaging the coupling device, such that the at least two male connecting portions are brought between the guiding edge, instead of trying to directly by precision obtain the matching of the at least two male connecting portions with the at least two female connecting portions. The guiding edge may further restrict lateral movement of the second connector part, such that the at least two male connecting portions of the second connector part may be securely rotated along the first connector surface until they engage with the at least two female connecting portions in less than one turn of the second connector part.
# medical_device_category: Gastroenterology-urology devices
# medical_device_type1: invasive NOTE non-invasive
# medical_device_type2: active NOTE non-active
# software: non-software
# measuring_function: no measuring function
# intended_user: enduser

# --------------------------------------------------
# EP3925641A1.pdf
# summary: An irrigation system, for irrigation of a hollow body organ, comprises a reusable part and a disposable part, the reusable part comprises a liquid container with a first opening and a first connection interface, the disposable part comprises a protective cover comprising a tubular part, a one-way valve in the tubular part, and a protective skirt connected to and encircling the tubular part, the tubular part is provided with a second connection interface which is releasably connectable to the first connection interface.
# key_technological_field: irrigation system, reusable part, disposable part, connection interface, one-way valve
# novelty_level: MEDIUM
# novelty_level_reason: The invention is novel because it combines a reusable part with a disposable part, and the disposable part has a protective skirt and a one-way valve, which are not present in the prior art.
# medical_device_category: Gastroenterology-urology devices
# medical_device_type1: invasive
# medical_device_type2: active
# software: non-software
# measuring_function: no measuring function
# intended_user: enduser

# --------------------------------------------------
# EP2671601A1.pdf
# summary: An irrigation system is disclosed, for use e.g. for rectal irrigation, and being suitable for self-administration of an irrigation liquid. The system comprises a reservoir for an irrigating liquid, such as water, a probe/catheter for arrangement in a user and a control unit for controlling the transfer of said irrigation liquid. Further, tubing is provided for establishing fluid communication between the reservoir, control unit and probe. The reservoir is arranged to exert a mechanical pressure on the irrigation liquid therein, thereby maintaining the irrigation liquid under pressure, and wherein the control unit comprises a control valve for releasing irrigation liquid from the reservoir for discharge through said probe.
# key_technological_field: irrigation system, reservoir, control unit, probe, irrigation liquid
# novelty_level: MEDIUM
# novelty_level_reason: The invention is novel in that it provides an irrigation system that is easy to use and cost-efficient to produce, and which is suitable for self-administration of the irrigation liquid.
# medical_device_category: Gastroenterology-urology devices
# medical_device_type1: invasive
# medical_device_type2: active
# software: non-software
# measuring_function: no measuring function NOTE measuring function
# intended_user: enduser

# --------------------------------------------------
# EP2810669A1.pdf
# summary: A collapsible container for medical use, comprising a flexible side wall, a rigid bottom portion and a rigid top portion, wherein the flexible side wall is arranged between the rigid bottom and top portions, and wherein the container is reversibly foldable and unfoldable, thereby being arrangeable in a compact state, in which the rigid top and bottom portions are arranged relatively closer to each other, and in an expanded state, in which the rigid top and bottom portions are arranged relatively further apart, respectively.
# key_technological_field: collapsible container, medical use, flexible side wall, rigid bottom portion, rigid top portion
# novelty_level: MEDIUM
# novelty_level_reason: The invention is a collapsible container for medical use, which is not a new concept, but the combination of a flexible side wall with rigid bottom and top portions, and the reversible foldability and unfoldability of the container, make it a novel invention.
# medical_device_category: Gastroenterology-urology devices
# medical_device_type1: non-invasive
# medical_device_type2: non-active NOTE active
# software: non-software
# measuring_function: no measuring function
# intended_user: enduser

# --------------------------------------------------
# EP3295977A1.pdf
# summary: An irrigation system and a method for controlling the flow rate of an irrigation liquid in an irrigation system are disclosed, wherein the flow rate is controlled by a double control system, comprising a first coarse control of the flow rate by controlling the electric pump, and a second finer control of the output flow rate by controlling degree of openness of the electrically operable valve.
# key_technological_field: Irrigation system, Flow rate control, Electric pump, Electrically operable valve, Double control system
# novelty_level: HIGH
# novelty_level_reason: The invention is highly novel as it provides a double control system for controlling the flow rate of an irrigation liquid in an irrigation system, comprising a first coarse control of the flow rate by controlling the electric pump, and a second finer control of the output flow rate by controlling degree of openness of the electrically operable valve.
# medical_device_category: Gastroenterology-urology devices
# medical_device_type1: invasive
# medical_device_type2: active
# software: non-software NOTE software
# measuring_function: no measuring function NOTE measuring function
# intended_user: enduser

# --------------------------------------------------


In [ ]:
# # MIXTRAL yes/no prompt 2 min/patent

# EP2679259A1.pdf
# summary: An irrigation system comprising a reservoir for an irrigating liquid, a probe for arrangement in a user, an electrical system, a control unit for controlling the transfer of irrigation liquid, and tubing providing fluid communication between the reservoir, control unit and probe, wherein the control unit comprises a waterproof housing enclosing at least the battery and the electrical system is galvanically isolated from the exterior of the irrigation system, and wherein the battery is chargeable through inductive charging.
# key_technological_field: irrigation system, waterproof housing, galvanic isolation, inductive charging
# novelty_level: MEDIUM
# novelty_level_reason: The irrigation system is not the first of its kind, but it has some unique features such as the waterproof housing and the galvanic isolation of the electrical system, which make it more robust and safe to use. Additionally, the battery is chargeable through inductive charging, which eliminates the need for electrical connections and further increases the safety of the system.
# medical_device_category: Gastroenterology-urology devices
# invasive: yes
# active: yes
# software: yes
# measuring_function: no NOTE yes
# intended_user: no

# --------------------------------------------------
# WO2014076653A1.pdf
# summary: A twist drill and bone tap each monitor torque while drilling or threading to assess jaw bone quality and a method for accessing bone quality prior to or while tapping into the bone during a dental implantation procedure.
# key_technological_field: torque monitoring, dental implantation, bone quality assessment, twist drill, bone tap
# novelty_level: MEDIUM
# novelty_level_reason: The invention is novel in that it provides a twist drill and bone tap that monitor torque while drilling or threading to assess jaw bone quality and a method for accessing bone quality prior to or while tapping into the bone during a dental implantation procedure.
# medical_device_category: Dental devices
# invasive: yes
# active: yes
# software: yes
# measuring_function: yes
# intended_user: yes

# --------------------------------------------------
# EP2671602A1.pdf
# summary: An irrigation system comprising a reservoir for an irrigating liquid, a probe for arrangement in a user, a first pump for directly or indirectly pumping irrigation liquid from the reservoir to the probe, a second pump for pumping a fluid to the balloon for inflation, a control unit for controlling the transfer of the irrigation liquid and the fluid, wherein the control unit further comprises a valve for releasing fluid from the balloon for deflation, the valve being provided with a manually operable balloon control element, and tubing providing fluid communication between the reservoir, control unit and probe; wherein the control unit and the first and second pumps are arranged as an integrated unitary component, and the manually operable balloon control element is arranged in the vicinity of the second pump.
# key_technological_field: irrigation system, balloon control, pump, reservoir, tubing
# novelty_level: MEDIUM
# novelty_level_reason: The invention is novel in that it provides an irrigation system with a control unit and pumps arranged as an integrated unitary component, and a manually operable balloon control element arranged in the vicinity of the second pump.
# medical_device_category: Gastroenterology-urology devices
# invasive: yes
# active: yes
# software: no
# measuring_function: no
# intended_user: no

# --------------------------------------------------
# EP2783724A1.pdf
# summary: A coupling device for a tubular arrangement for medical use comprising a first connector part and a second connector part, wherein one of said connector parts comprises an engagement/disengagement member for at least assisting in engaging and disengaging the engagement between said connector parts, and wherein the other of said connector parts comprises a locking element arranged to rotatably engage with the engagement/disengagement member, wherein the first connector part comprises at least two female connecting portions arranged in a first connector surface and a guiding edge protruding from the first connector surface arranged around the at least two female connecting portions, and the second connector part comprises at least two male connecting portions arranged in a second connector surface, and wherein at least one of the first connector part and the second connector part comprises a locking element arranged to rotatably engage with the engagement/disengagement member.
# key_technological_field: coupling device, tubular arrangement, medical use, engagement/disengagement member, locking element
# novelty_level: MEDIUM
# novelty_level_reason: The invention is novel in that it provides a coupling device for a tubular arrangement for medical use with a guiding edge that facilitates engagement of the coupling device, by forming a recess in the connection interface. The guiding edge enables reduction of the need for precision when engaging the coupling device, such that the at least two male connecting portions are brought between the guiding edge, instead of trying to directly by precision obtain the matching of the at least two male connecting portions with the at least two female connecting portions. The guiding edge may further restrict lateral movement of the second connector part, such that the at least two male connecting portions of the second connector part may be securely rotated along the first connector surface until they engage with the at least two female connecting portions in less than one turn of the second connector part.
# medical_device_category: Gastroenterology-urology devices
# invasive: yes NOTE no
# active: yes NOTE no
# software: no
# measuring_function: no
# intended_user: no

# --------------------------------------------------
# EP3925641A1.pdf
# summary: An irrigation system, for irrigation of a hollow body organ, comprises a reusable part and a disposable part, the reusable part comprises a liquid container with a first opening and a first connection interface, the disposable part comprises a protective cover comprising a tubular part, a one-way valve in the tubular part, and a protective skirt connected to and encircling the tubular part, the tubular part is provided with a second connection interface which is releasably connectable to the first connection interface.
# key_technological_field: irrigation system, reusable part, disposable part, connection interface, one-way valve
# novelty_level: MEDIUM
# novelty_level_reason: The invention is novel because it combines a reusable part with a disposable part, and the disposable part has a protective skirt and a one-way valve, which are not present in the prior art.
# medical_device_category: Gastroenterology-urology devices
# invasive: yes
# active: yes
# software: no
# measuring_function: no
# intended_user: no

# --------------------------------------------------
# EP2671601A1.pdf
# summary: An irrigation system is disclosed, for use e.g. for rectal irrigation, and being suitable for self-administration of an irrigation liquid. The system comprises a reservoir for an irrigating liquid, such as water, a probe/catheter for arrangement in a user and a control unit for controlling the transfer of said irrigation liquid. Further, tubing is provided for establishing fluid communication between the reservoir, control unit and probe. The reservoir is arranged to exert a mechanical pressure on the irrigation liquid therein, thereby maintaining the irrigation liquid under pressure, and wherein the control unit comprises a control valve for releasing irrigation liquid from the reservoir for discharge through said probe.
# key_technological_field: irrigation system, reservoir, control unit, probe, irrigation liquid
# novelty_level: MEDIUM
# novelty_level_reason: The invention is novel in that it provides an irrigation system that is easy to use and cost-efficient to produce, and which is suitable for self-administration of the irrigation liquid.
# medical_device_category: Gastroenterology-urology devices
# invasive: yes
# active: yes
# software: no
# measuring_function: no NOTE yes
# intended_user: no

# --------------------------------------------------
# EP2810669A1.pdf
# summary: A collapsible container for medical use, comprising a flexible side wall, a rigid bottom portion and a rigid top portion, wherein the flexible side wall is arranged between the rigid bottom and top portions, and wherein the container is reversibly foldable and unfoldable, thereby being arrangeable in a compact state, in which the rigid top and bottom portions are arranged relatively closer to each other, and in an expanded state, in which the rigid top and bottom portions are arranged relatively further apart, respectively.
# key_technological_field: collapsible container, medical use, flexible side wall, rigid bottom portion, rigid top portion
# novelty_level: MEDIUM
# novelty_level_reason: The invention is a collapsible container for medical use, which is not a new concept, but the combination of a flexible side wall with rigid bottom and top portions, and the reversible foldability and unfoldability of the container, make it a novel invention.
# medical_device_category: Gastroenterology-urology devices
# invasive: yes NOTE no
# active: yes
# software: no
# measuring_function: no
# intended_user: yes NOTE no
 
# --------------------------------------------------
# EP3295977A1.pdf
# summary: An irrigation system and a method for controlling the flow rate of an irrigation liquid in an irrigation system are disclosed, wherein the flow rate is controlled by a double control system, comprising a first coarse control of the flow rate by controlling the electric pump, and a second finer control of the output flow rate by controlling degree of openness of the electrically operable valve.
# key_technological_field: Irrigation system, Flow rate control, Electric pump, Electrically operable valve, Double control system
# novelty_level: HIGH
# novelty_level_reason: The invention is highly novel as it provides a double control system for controlling the flow rate of an irrigation liquid in an irrigation system, comprising a first coarse control of the flow rate by controlling the electric pump, and a second finer control of the output flow rate by controlling degree of openness of the electrically operable valve.
# medical_device_category: Gastroenterology-urology devices
# invasive: yes
# active: yes
# software: yes
# measuring_function: yes
# intended_user: no

# --------------------------------------------------


#### Summary:
mistral choice prompt: 9 mistakes / 48 \
mistral yes/no prompt: 13 mistakes / 48 \
mixtral choice prompt: 9 mistakes / 48 \
mixtral yes/no prompt: 7 mistakes / 48 -> 15%

---
---

In [ ]:
# example to generate Pydantic model on the fly based on fields in a dict
from pydantic import create_model

metadata_dict = {}
for key in ArticleMetadata.model_fields.keys():
    metadata_dict[key] = output[key]

DynamicUser = create_model('NodeMetadata_generated', **{k: (type(v), ...) for k, v in metadata_dict.items()})
output_model_generated = DynamicUser(**metadata_dict)
print(type(output_model_generated))